In [1]:
from keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, Lambda
from keras import backend as K
from sklearn.datasets import fetch_olivetti_faces
import numpy as np

In [2]:
def create_model():
    inputs = Input((64, 64, 1))
    x = Conv2D(96, (11, 11), padding="same", activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (5, 5), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(384, (3, 3), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    pooledOutput = GlobalAveragePooling2D()(x)
    pooledOutput = Dense(1024)(pooledOutput)
    outputs = Dense(128)(pooledOutput)

    model = Model(inputs, outputs)
    return model

In [3]:
feature_extractor = create_model()
imgA = Input(shape=(64, 64, 1))
imgB = Input(shape=(64, 64, 1))
featA = feature_extractor(imgA)
featB = feature_extractor(imgB)

In [4]:
def euclidean_distance(vectors):
    (featA, featB) = vectors
    sum_squared = K.sum(K.square(featA - featB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

distance = Lambda(euclidean_distance)([featA, featB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)

In [5]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [6]:
def generate_train_image_pairs(images_dataset, labels_dataset):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                      [index for index, curr_label in enumerate(labels_dataset) if label == curr_label])
    
    pair_images = []
    pair_labels = []
    for index, image in enumerate(images_dataset):
        pos_indices = label_wise_indices.get(labels_dataset[index])
        pos_image = images_dataset[np.random.choice(pos_indices)]
        pair_images.append((image, pos_image))
        pair_labels.append(1)

        neg_indices = np.where(labels_dataset != labels_dataset[index])
        neg_image = images_dataset[np.random.choice(neg_indices[0])]
        pair_images.append((image, neg_image))
        pair_labels.append(0)
    return np.array(pair_images), np.array(pair_labels)

In [7]:
image_data = fetch_olivetti_faces()
images_dataset = image_data.images
labels_dataset = image_data.target

In [8]:
images_pair, labels_pair = generate_train_image_pairs(images_dataset, labels_dataset)
history = model.fit([images_pair[:, 0], images_pair[:, 1]], labels_pair[:],validation_split=0.1,batch_size=64,epochs=100)

Epoch 1/100
12/12 [==============================] - 327s 26s/step - loss: 0.6951 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/100
12/12 [==============================] - 281s 24s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/100
12/12 [==============================] - 279s 23s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/100
12/12 [==============================] - 270s 22s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/100
12/12 [==============================] - 274s 23s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/100
12/12 [==============================] - 293s 25s/step - loss: 0.6936 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/100
12/12 [==============================] - 305s 24s/step - loss: 0.6936 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 58/100
12/12 [==============================] - 279s 23s/step - loss: 0.6933 - accuracy: 0.5056 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 59/100
12/12 [==============================] - 273s 23s/step - loss: 0.6935 - accuracy: 0.5097 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 60/100
12/12 [==============================] - 276s 23s/step - loss: 0.6930 - accuracy: 0.5069 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 61/100
12/12 [==============================] - 272s 23s/step - loss: 0.6933 - accuracy: 0.4833 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 62/100
12/12 [==============================] - 275s 23s/step - loss: 0.6931 - accuracy: 0.4931 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 63/100
12/12 [==============================] - 277s 23s/step - loss: 0.6932 - accuracy: 0.4889 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 64/100
12/12 [==============================] - 299s 25s/step - loss: 0.6933 - accuracy: 0.5083 - val_loss: 0.6933 - val_accuracy:

In [9]:
def generate_test_image_pairs(images_dataset, labels_dataset, image):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label, [index for index, curr_label in enumerate(labels_dataset) if label == curr_label])
    pair_images = []
    pair_labels = []
    for label, indices_for_label in label_wise_indices.items():
        test_image = images_dataset[np.random.choice(indices_for_label)]
        pair_images.append((image, test_image))
        pair_labels.append(label)
    return np.array(pair_images), np.array(pair_labels)

In [10]:
image = images_dataset[92] # a random image as test image
test_image_pairs, test_label_pairs = generate_test_image_pairs(images_dataset, labels_dataset, image)
for index, pair in enumerate(test_image_pairs):
    pair_image1 = np.expand_dims(pair[0], axis=-1)
    pair_image1 = np.expand_dims(pair_image1, axis=0)
    pair_image2 = np.expand_dims(pair[1], axis=-1)
    pair_image2 = np.expand_dims(pair_image2, axis=0)
    prediction = model.predict([pair_image1, pair_image2])[0][0]

1/1 [==============================] - 0s 141ms/step
